# Composite Pattern

- __Type:__ Structural
- __Popularity: ★★★★☆__
- __Complexity: ★★★☆☆__

### Intent:
__Composite Pattern__ is a structural design pattern that lets you compose objects into tree structures to represent part-whole hierarchies. It allows clients to treat individual objects and compositions of objects uniformly.

### Problem:
When working with tree-structured data, we often need to distinguish between a leaf-node and a branch. If client code needs to check the object type before performing operations, it creates tight coupling and makes the code hard to extend when new component types are added. Specifically:

- The need to treat both simple (leaf) elements and complex (composite) elements uniformly
- Different processing required for leaf and composite elements creates complexity
- Adding new component types requires changes throughout the codebase
- Difficulty in representing part-whole hierarchies that require similar operations on both parts and wholes

### Solution:
The Composite pattern solves this by:

1. Defining a common interface/abstract class for all components (both leaf and composite)
2. Implementing leaf objects that represent end nodes with no children
3. Implementing composite objects that can contain other components (both leaf and composite)
4. Having composite objects delegate operations to their children

The pattern enables clients to work with complex hierarchies through a simple, unified interface. It hides the complex tree structure and lets client code work with all objects - whether simple or complex - through the same interface.

### Diagram:

```mermaid
classDiagram
    class Component {
        +operation()
        +add(component)
        +remove(component)
        +is_composite()
    }
    class Leaf {
        +operation()
    }
    class Composite {
        -children: List[Component]
        +operation()
        +add(component)
        +remove(component)
        +is_composite()
    }
    
    Component <|-- Leaf
    Component <|-- Composite
    Composite o-- Component
```

## Example 1: Basic Composite Pattern

In [1]:
from abc import ABC, abstractmethod
from typing import List


class Component(ABC):
    """Base Component class declares operations common to both simple and complex objects"""

    @property
    def parent(self):
        return self._parent

    @parent.setter
    def parent(self, parent):
        self._parent = parent

    def add(self, component) -> None:
        """Optionally implemented in subclasses"""
        pass

    def remove(self, component) -> None:
        """Optionally implemented in subclasses"""
        pass

    def is_composite(self) -> bool:
        """Helps distinguish between leaves and composites"""
        return False

    @abstractmethod
    def operation(self) -> str:
        """Must be implemented by all concrete components"""
        pass


class Leaf(Component):
    """Leaf class represents end objects with no children"""

    def operation(self) -> str:
        return "Leaf"


class Composite(Component):
    """Composite class represents complex components that may have children"""

    def __init__(self) -> None:
        self._children: List[Component] = []

    def add(self, component: Component) -> None:
        self._children.append(component)
        component.parent = self

    def remove(self, component: Component) -> None:
        self._children.remove(component)
        component.parent = None

    def is_composite(self) -> bool:
        return True

    def operation(self) -> str:
        """Calls the operation method on all child components and combines results"""
        results = []
        for child in self._children:
            results.append(child.operation())

        return f"Branch({'+'.join(results)})"

### Usage

In [2]:
def client_code(component: Component) -> None:
    """Client code can work with any component"""
    print(f"RESULT: {component.operation()}")


def client_code_2(component1: Component, component2: Component) -> None:
    """Client code can add components to composites"""
    if component1.is_composite():
        component1.add(component2)

    print(f"RESULT: {component1.operation()}")


if __name__ == "__main__":
    # Using a simple component
    simple = Leaf()
    print("Client: I've got a simple component:")
    client_code(simple)
    print("\n")

    # Creating a tree structure
    tree = Composite()

    branch1 = Composite()
    branch1.add(Leaf())
    branch1.add(Leaf())

    branch2 = Composite()
    branch2.add(Leaf())

    # Adding branches to the tree
    tree.add(branch1)
    tree.add(branch2)

    print("Client: Now I've got a composite tree:")
    client_code(tree)
    print("\n")

    print("Client: I don't need to check the classes even when managing the tree:")
    client_code_2(tree, simple)

Client: I've got a simple component:
RESULT: Leaf


Client: Now I've got a composite tree:
RESULT: Branch(Branch(Leaf+Leaf)+Branch(Leaf))


Client: I don't need to check the classes even when managing the tree:
RESULT: Branch(Branch(Leaf+Leaf)+Branch(Leaf)+Leaf)


## Example 2: File System Implementation

In [3]:
from abc import ABC, abstractmethod
from typing import List


class FileSystemComponent(ABC):
    """Abstract base class for all file system components"""

    def __init__(self, name: str):
        self.name = name

    @abstractmethod
    def get_size(self) -> int:
        """Returns size of the file system component"""
        pass

    @abstractmethod
    def print_details(self, indent: str = "") -> None:
        """Prints the details of the file system component"""
        pass

    def add(self, component) -> None:
        """Default implementation - only directories can add components"""
        raise NotImplementedError("Cannot add to a non-directory component")

    def remove(self, component) -> None:
        """Default implementation - only directories can remove components"""
        raise NotImplementedError("Cannot remove from a non-directory component")


class File(FileSystemComponent):
    """Leaf node representing a file"""

    def __init__(self, name: str, size: int):
        super().__init__(name)
        self._size = size

    def get_size(self) -> int:
        return self._size

    def print_details(self, indent: str = "") -> None:
        print(f"{indent}File: {self.name}, Size: {self._size} bytes")


class Directory(FileSystemComponent):
    """Composite node representing a directory that can contain files and subdirectories"""

    def __init__(self, name: str):
        super().__init__(name)
        self._children: List[FileSystemComponent] = []

    def add(self, component: FileSystemComponent) -> None:
        self._children.append(component)

    def remove(self, component: FileSystemComponent) -> None:
        self._children.remove(component)

    def get_size(self) -> int:
        """Calculate total size by summing all children"""
        total_size = 0
        for child in self._children:
            total_size += child.get_size()
        return total_size

    def print_details(self, indent: str = "") -> None:
        print(f"{indent}Directory: {self.name}, Size: {self.get_size()} bytes")
        # Print all children with increased indentation
        for child in self._children:
            child.print_details(indent + "  ")

### Usage

In [4]:
if __name__ == "__main__":
    # Create file system structure
    root = Directory("root")

    # Add some files to root
    root.add(File("config.json", 120))
    root.add(File("readme.md", 350))

    # Create a subdirectory
    docs = Directory("docs")
    docs.add(File("manual.pdf", 2200))
    docs.add(File("reference.txt", 800))

    # Create another level of subdirectory
    images = Directory("images")
    images.add(File("logo.png", 450))
    images.add(File("banner.jpg", 1800))

    # Add subdirectories to their parent directories
    docs.add(images)
    root.add(docs)

    # Add another top-level directory
    src = Directory("src")
    src.add(File("main.py", 540))
    src.add(File("utils.py", 320))
    root.add(src)

    # Print the file system structure
    root.print_details()

    # Calculate and print the size of a specific directory
    print(f"\nSize of docs directory: {docs.get_size()} bytes")

    # Remove a component and see the updated structure
    print("\nRemoving images directory...")
    docs.remove(images)
    root.print_details()

Directory: root, Size: 6580 bytes
  File: config.json, Size: 120 bytes
  File: readme.md, Size: 350 bytes
  Directory: docs, Size: 5250 bytes
    File: manual.pdf, Size: 2200 bytes
    File: reference.txt, Size: 800 bytes
    Directory: images, Size: 2250 bytes
      File: logo.png, Size: 450 bytes
      File: banner.jpg, Size: 1800 bytes
  Directory: src, Size: 860 bytes
    File: main.py, Size: 540 bytes
    File: utils.py, Size: 320 bytes

Size of docs directory: 5250 bytes

Removing images directory...
Directory: root, Size: 4330 bytes
  File: config.json, Size: 120 bytes
  File: readme.md, Size: 350 bytes
  Directory: docs, Size: 3000 bytes
    File: manual.pdf, Size: 2200 bytes
    File: reference.txt, Size: 800 bytes
  Directory: src, Size: 860 bytes
    File: main.py, Size: 540 bytes
    File: utils.py, Size: 320 bytes


### Real-world analogies:

1. Organization Hierarchies:
   - A corporation is composed of departments, which contain employees
   - The CEO can request a report from the entire company or a specific department
   - Each department compiles reports from its employees, mirroring how composite objects delegate to their children

2. File Systems:
   - A file system contains both files (leaves) and directories (composites)
   - Both files and directories share common operations like deletion, moving, or calculating size
   - When calculating a directory's size, it sums the sizes of all contained files and subdirectories

### When to use:

- When you need to represent part-whole hierarchies of objects
- When you want clients to be able to ignore the differences between compositions of objects and individual objects
- When the core model can be represented as a tree structure
- When you want to be able to treat all objects in the composite structure uniformly
- When adding new types of components should not require changes to existing code

### Python-specific implementation notes:

- Python's duck typing allows more flexible implementations than statically typed languages
- The `abc` module with its `ABC` class and `abstractmethod` decorator helps enforce the component interface
- Python's built-in methods like `__getitem__`, `__iter__` can enhance composite implementations
- Python's list and dictionary comprehensions make it easy to process children in composites
- Python's `os` and `pathlib` modules already implement composite pattern for file system operations

### Related patterns:

- Decorator: Often used in conjunction with Composite to add responsibilities to objects
- Iterator: Can be used to traverse composite structures without exposing their internal representation
- Visitor: Can be used to perform operations over a Composite structure
- Builder: Can be used to create complex Composite structures step by step
- Chain of Responsibility: Often the composite parent passes requests along the chain of child components